In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/MN-20-Credit/Aerial-YOLO-DOTA/src

In [ ]:
!pip -q install ultralytics

In [ ]:
import os, glob, random, math, copy, cv2, numpy as np, shutil
from tqdm import trange
import torch
from ultralytics import YOLO

In [ ]:
MODEL_PATH = "/content/drive/MyDrive/MN-20-Credit/models/yolov9/best.pt"
IMG_DIR    = "/content/drive/MyDrive/MN-20-Credit/dota-yolo/images/val"   # folder of images to attack
OUT_DIR    = "/content/drive/MyDrive/MN-20-Credit/dota-yolo/patch_attack_out" # patched images + learned patch

# Recreate output folder
PATCHED_DIR = os.path.join(OUT_DIR, "patched_val")
if os.path.exists(PATCHED_DIR):
    shutil.rmtree(PATCHED_DIR)
os.makedirs(PATCHED_DIR, exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)

In [ ]:
yolo = YOLO(MODEL_PATH)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
IMG_SIZE = 1024  # we resize images to this canvas for patch learning

# -------------------- EoT PARAMS --------------------
BATCH_SIZE     = 6        # images per iteration
MAX_ITERS      = 2000      # total iterations (increase for stronger patch)
EVAL_SAMPLES   = 64        # images used to compute/report score occasionally
CONF_THRES     = 0.001     # detection threshold for scoring
IOU_THRES      = 0.5

PATCH_REL_SIZE = 0.3    # patch height as fraction of image side (width set by aspect)
PATCH_ASPECT   = 1.0      # patch_w = PATCH_ASPECT * patch_h
OPACITY_RANGE  = (0.95, 1.0)   # alpha for paste
ROT_MAX_DEG    = 20.0     # random rotation range [-ROT_MAX_DEG, +ROT_MAX_DEG]
SCALE_RANGE    = (0.9, 1.1)   # multiplicative scale *after* PATCH_REL_SIZE base
MUT_STROKES    = 10        # how many rectangles to mutate per proposal
MUT_RECT_FRAC  = 0.35     # each rect size as fraction of patch dims (max)

# # Untargeted: reduce any detections; Targeted (optional): only count detections for this class id
TARGET_CLASS_ID = None     # e.g., 8 for 'plane', or None for untargeted

# BATCH_SIZE     = 8
# MAX_ITERS      = 600
# EVAL_SAMPLES   = 64
# CONF_THRES     = 0.001   # search objective more sensitive
# IOU_THRES      = 0.5

# PATCH_REL_SIZE = 0.40    # larger, more disruptive
# PATCH_ASPECT   = 1.5
# OPACITY_RANGE  = (0.98, 1.00)
# ROT_MAX_DEG    = 15.0    # slightly tighter to avoid over-regularizing
# SCALE_RANGE    = (0.85, 1.15)
# MUT_STROKES    = 16
# MUT_RECT_FRAC  = 0.45

random.seed(0); np.random.seed(0)

In [ ]:
# -------------------- HELPERS --------------------
def list_images(folder):
    exts = (".jpg",".jpeg",".png",".bmp",".tif",".tiff")
    return sorted([p for p in glob.glob(os.path.join(folder, "*")) if p.lower().endswith(exts)])

def load_image_1024(path):
    img = cv2.imread(path)  # BGR
    if img is None:
        raise FileNotFoundError(path)
    return cv2.resize(img, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_LINEAR)

def random_patch_init(h, w):
    # start with random noise; you can also try mid-gray np.full(...)
    return np.random.randint(0, 256, (h, w, 3), dtype=np.uint8)

def mutate_patch(patch):
    """Return a mutated copy of the patch by 'painting' a few random rectangles."""
    ph, pw = patch.shape[:2]
    cand = patch.copy()
    for _ in range(MUT_STROKES):
        rw = max(1, int(random.uniform(0.05, MUT_RECT_FRAC) * pw))
        rh = max(1, int(random.uniform(0.05, MUT_RECT_FRAC) * ph))
        x0 = random.randint(0, max(0, pw - rw))
        y0 = random.randint(0, max(0, ph - rh))
        color = np.random.randint(0, 256, (1,1,3), dtype=np.uint8)
        cand[y0:y0+rh, x0:x0+rw] = color
    return cand

def transform_patch(patch):
    """Random scale+rotate the patch and produce a mask."""
    ph, pw = patch.shape[:2]
    # base scale from relative size + jitter
    s = random.uniform(*SCALE_RANGE)
    new_h = max(2, int(ph * s))
    new_w = max(2, int(pw * s))
    pat_rs = cv2.resize(patch, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

    # rotate around center
    angle = random.uniform(-ROT_MAX_DEG, ROT_MAX_DEG)
    M = cv2.getRotationMatrix2D((new_w/2, new_h/2), angle, 1.0)
    pat_rot = cv2.warpAffine(pat_rs, M, (new_w, new_h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)

    # rectangular mask (1 where patch exists). Could soften edges if you like.
    mask = np.ones((new_h, new_w, 1), dtype=np.float32)
    # random opacity per application
    alpha = random.uniform(*OPACITY_RANGE)
    mask *= alpha
    return pat_rot, mask

def paste_patch(img, patch_t, mask_t, x, y):
    """Paste transformed patch at (x,y) top-left with mask alpha blending."""
    H, W = img.shape[:2]
    h, w = patch_t.shape[:2]
    x = int(np.clip(x, 0, W-1))
    y = int(np.clip(y, 0, H-1))
    x2 = min(W, x + w)
    y2 = min(H, y + h)
    pw = x2 - x
    ph = y2 - y
    if pw <= 0 or ph <= 0:
        return img  # nothing to paste

    out = img.copy()
    # crop if patch extends beyond borders
    patch_c = patch_t[:ph, :pw]
    mask_c  = mask_t[:ph, :pw]
    # alpha blend: out = mask*patch + (1-mask)*img
    out[y:y+ph, x:x+pw] = (mask_c * patch_c + (1.0 - mask_c) * out[y:y+ph, x:x+pw]).astype(np.uint8)
    return out

def random_place_coords(img, patch_t):
    H, W = img.shape[:2]
    h, w = patch_t.shape[:2]
    # allow full range (may clip at borders in paste)
    x = random.randint(0, max(0, W - 1))
    y = random.randint(0, max(0, H - 1))
    return x, y

def score_images(images_bgr):
    """
    Lower is better (we want to suppress detections).
    We use sum of detection confidences (or count) across the batch.
    """
    total = 0.0
    for img in images_bgr:
        # Ultralytics accepts numpy BGR; keep low conf to "see" detections
        r = yolo.predict(source=img, imgsz=IMG_SIZE, conf=CONF_THRES, iou=IOU_THRES, verbose=False)[0]
        if r.boxes is None or len(r.boxes) == 0:
            continue
        scores = r.boxes.conf.cpu().numpy()
        labels = r.boxes.cls.cpu().numpy().astype(int)
        if TARGET_CLASS_ID is not None:
            scores = scores[labels == TARGET_CLASS_ID]
        # use sum of confidences (you can try len(scores) for pure count)
        total += float(scores.sum())
    return total

def batch_with_patch(img_paths, patch, batch_size=BATCH_SIZE):
    """Draw a random batch, apply EoT transforms & place the patch."""
    sel = random.sample(img_paths, k=min(batch_size, len(img_paths)))
    out_imgs = []
    for p in sel:
        img = load_image_1024(p)
        pat_t, mask_t = transform_patch(patch)
        x, y = random_place_coords(img, pat_t)
        img_p = paste_patch(img, pat_t, mask_t, x, y)
        out_imgs.append(img_p)
    return out_imgs

In [ ]:
# -------------------- MAIN OPTIMIZATION LOOP --------------------
all_imgs = list_images(IMG_DIR)
assert len(all_imgs) > 0, "No images found in IMG_DIR"

# initial patch size (in pixels) from relative size
patch_h = int(PATCH_REL_SIZE * IMG_SIZE)
patch_w = int(PATCH_ASPECT * patch_h)
best_patch = random_patch_init(patch_h, patch_w)

# baseline score (no patch) for reference
baseline_imgs = [load_image_1024(p) for p in random.sample(all_imgs, min(EVAL_SAMPLES, len(all_imgs)))]
baseline_score = score_images(baseline_imgs)
print(f"Baseline score (no patch) on {len(baseline_imgs)} imgs: {baseline_score:.3f}")

# score with initial random patch
test_imgs = batch_with_patch(all_imgs, best_patch, batch_size=EVAL_SAMPLES)
best_score = score_images(test_imgs)
print(f"Initial patch score: {best_score:.3f} (lower is better)")

for it in trange(MAX_ITERS, desc="Optimizing patch"):
    candidate = mutate_patch(best_patch)
    imgs = batch_with_patch(all_imgs, candidate, batch_size=BATCH_SIZE)
    cand_score = score_images(imgs)
    # Accept if better (you can add simulated annealing noise if you want)
    if cand_score < best_score:
        best_patch = candidate
        best_score = cand_score

    # Occasionally report validation score on a fresh mini-sample
    if (it+1) % 20 == 0:
        val_imgs = batch_with_patch(all_imgs, best_patch, batch_size=EVAL_SAMPLES)
        val_score = score_images(val_imgs)
        print(f"\nIter {it+1}: best_score={best_score:.3f}  val_score={val_score:.3f}")

# Save learned patch
patch_path = os.path.join(OUT_DIR, "universal_patch.png")
cv2.imwrite(patch_path, best_patch)
print("Saved patch:", patch_path)

Baseline score (no patch) on 64 imgs: 4929.622
Initial patch score: 3488.599 (lower is better)


Optimizing patch:   1%|          | 20/2000 [01:23<6:21:17, 11.55s/it]


Iter 20: best_score=95.651  val_score=4352.267


Optimizing patch:   2%|▏         | 30/2000 [01:51<2:01:43,  3.71s/it]


KeyboardInterrupt: 

In [ ]:
# -------------------- APPLY PATCH TO A FOLDER --------------------
import shutil, os

PATCHED_DIR = os.path.join(OUT_DIR, "patched_val")
if os.path.exists(PATCHED_DIR):
    shutil.rmtree(PATCHED_DIR)   # ⚠️ deletes everything inside

os.makedirs(PATCHED_DIR, exist_ok=True)

for p in all_imgs:
    img = load_image_1024(p)
    pat_t, mask_t = transform_patch(best_patch)
    x, y = random_place_coords(img, pat_t)
    img_p = paste_patch(img, pat_t, mask_t, x, y)
    outp = os.path.join(PATCHED_DIR, os.path.basename(p))
    cv2.imwrite(outp, img_p)

print("Patched images written to:", PATCHED_DIR)

In [ ]:
# --- Colab-ready: evaluate mAP on your patched images ---
import os, glob, shutil, yaml
from ultralytics import YOLO

# Paths
MODEL_PATH = "/content/drive/MyDrive/MN-20-Credit/models/yolov9/best.pt"
ROOT       = "/content/drive/MyDrive/MN-20-Credit/dota-yolo"              # dataset root (has images/ and labels/)
OUT_DIR    = "/content/drive/MyDrive/MN-20-Credit/dota-yolo/patch_attack_out/patched_val"  # your patched outputs

# Destination split under the dataset root
dst_img_dir = os.path.join(ROOT, "images", "val_patched")
dst_lbl_dir = os.path.join(ROOT, "labels", "val_patched")
src_lbl_dir = os.path.join(ROOT, "labels", "val")

os.makedirs(dst_img_dir, exist_ok=True)

# (Optional) clear previous val_patched to avoid stale files
for p in glob.glob(os.path.join(dst_img_dir, "*")):
    try: os.remove(p)
    except: pass

# Copy patched images into images/val_patched, renaming to original basenames
img_exts = {".jpg",".jpeg",".png",".bmp",".tif",".tiff"}
copied = 0
for p in glob.glob(os.path.join(OUT_DIR, "*")):
    ext = os.path.splitext(p)[1].lower()
    if ext not in img_exts:
        continue
    stem = os.path.splitext(os.path.basename(p))[0]
    # If files are like P0017_patched.png, strip the suffix so labels match P0017.txt
    if stem.endswith("_patched"):
        stem = stem[:-8]
    outp = os.path.join(dst_img_dir, stem + ext)
    shutil.copy2(p, outp)
    copied += 1

print(f"Copied {copied} patched images to: {dst_img_dir}")

# Make labels/val_patched point to labels/val (symlink preferred; copy fallback)
if os.path.islink(dst_lbl_dir) or os.path.exists(dst_lbl_dir):
    pass
else:
    try:
        os.symlink(src_lbl_dir, dst_lbl_dir, target_is_directory=True)
        print("Symlinked labels:", dst_lbl_dir, "->", src_lbl_dir)
    except Exception as e:
        print("Symlink failed; copying labels instead:", e)
        shutil.copytree(src_lbl_dir, dst_lbl_dir)

# Class names (same order you trained)
NAMES = [
  "baseball-diamond","basketball-court","bridge","container-crane","ground-track-field",
  "harbor","helicopter","large-vehicle","plane","roundabout","ship","small-vehicle",
  "soccer-ball-field","storage-tank","swimming-pool","tennis-court"
]

# Write a tiny YAML that points val to images/val_patched
DATA_YAML_PATCHED = "/content/dota_patched.yaml"
cfg = {
    "path": ROOT,
    "train": "images/train",         # harmless here
    "val":   "images/val_patched",   # <-- patched split
    "names": {i:n for i,n in enumerate(NAMES)},
    "nc": len(NAMES)
}
with open(DATA_YAML_PATCHED, "w") as f:
    yaml.safe_dump(cfg, f, sort_keys=False)
print("Wrote", DATA_YAML_PATCHED)

# Run evaluation with the same settings you used for clean metrics
model = YOLO(MODEL_PATH)
metrics = model.val(
    data=DATA_YAML_PATCHED,
    split="val",
    device=0,
    batch=4,       # drop to 2 or 1 if OOM
    imgsz=896,     # keep identical to clean run
    half=True,
    amp=True,
    workers=2,
    conf=0.001,
    iou=0.5,
    verbose=False,
    plots=False
)

print("\n=== Patched metrics ===")
print("mAP@0.50      :", metrics.box.map50)
print("mAP@0.50:0.95 :", metrics.box.map)
print("Precision     :", metrics.box.mp)
print("Recall        :", metrics.box.mr)
